# **1. 단순 선형 회귀 분석**
- 전복의 나이를 예측하는 선형회귀모델을 생성하세요.
- 전복의 ‘성별’, ‘키’, ‘지름’, ‘높이’, ‘전체무게’, ‘몸통무게’, ‘내장무게’, ‘껍질무게’를 이용해 ‘껍질의 고리 수’를 예측한 뒤, **예측된 ‘껍질의 고리 수’에 1.5를 더하면 전복의 나이**가 됩니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# 기본 모듈 불러오기
import numpy as np
import pandas as pd

**1) 데이터 load 및 변형**

In [21]:
# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/ESAA/세션/abalone.csv")
data.head()
print(data.shape)

# 성별 M은 Male, F는 Female, I는 Infant 이므로 따로 열 만들기
for label in "MFI":
    data[label] = data["Sex"] == label
data.drop('Sex', axis=1, inplace=True)

(4177, 9)


**2) X, y 선택**
: y는 Rings열, X는 Rings열을 제외한 나머지를 선택하되 전부 실수가 되도록 한다.

In [22]:
# X,y 데이터 선택
y =data['Rings']
X = data.drop('Rings', axis=1)
X = X.astype(float)

 **3) train/test set 분리**

In [23]:
# 필요한 모듈 불러오기
import sklearn
from sklearn.model_selection import train_test_split

In [27]:
# train과 test set 분리 (train:test = 7:3 비율로)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

**4) 선형회귀모델 생성, 모델 예측치 구하기**

In [28]:
#필요한 모듈 불러오기
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [29]:
#선형회귀모델 생성 및 훈련
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [30]:
# 모델 예측치 구하기
y_preds = lr.predict(X_test)
# 모델 예측치를 활용해 최종적으로 전복의 나이를 예측
age_preds = y_preds + 1.5
age_preds

array([14.703125, 10.671875, 11.8125  , ..., 10.75    , 20.265625,
       12.515625])

**5) 모델 평가: MSE, RMSE, R2 score, corr 구하기**

In [31]:
#필요한 모듈 불러오기
import sklearn
from sklearn.metrics import mean_squared_error, r2_score

- MSE, RMSE

In [32]:
#mse, rmse
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
print(mse, rmse)

5.004778653620913 2.237136261746457


- R2 score

In [33]:
#R2 score 측정
r2 = r2_score(y_test, y_preds)
r2

0.5252988826334977

- 회귀 절편값

In [34]:
#회귀 절편 값
lr.intercept_

-75665726805056.67

- 회귀 계수 값

In [36]:
#회귀 계수 값
lr.coef_

array([-2.13005808e-01,  1.14784137e+01,  7.05311543e+00,  9.02028533e+00,
       -1.99895043e+01, -1.07197065e+01,  9.67580976e+00,  7.56657268e+13,
        7.56657268e+13,  7.56657268e+13])

- 상관계수

Hint: corr 함수 이용.

In [41]:
# 상관계수 구하기
corr = np.corrcoef(y_test, y_preds)
corr

array([[1.        , 0.72504324],
       [0.72504324, 1.        ]])

In [42]:
corr[0,1]**2

0.5256877004688808

In [46]:
df = pd.DataFrame({'y_test':y_test, 'y_preds':y_preds})
corr = df['y_test'].corr(df['y_preds'])
corr

0.7250432404132051

In [49]:
corr**2

0.5256877004688808

# **2. Polynomial features**

In [43]:
# PolynomialFeatures 라이브러리 호출
from sklearn.preprocessing import PolynomialFeatures

In [15]:
# 임의 데이터 생성

X = np.arange(6).reshape(3, 2)

df =  pd.DataFrame(X)
df.columns = ['x_1','x_2']
df

,x_1,x_2
0,0,1
1,2,3
2,4,5


In [18]:
# 차원은 2로 설정
# fit_transform 메소드를 통해 데이터 변환
# PolynomialFeatures로 변환 된 데이터를 데이터 프레임 형태로 변환
ploy = PolynomialFeatures(degree=2)
df_poly = pd.DataFrame(ploy.fit_transform(df))
df_poly

,0,1,2,3,4,5
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0


In [19]:
# df_poly의 컬럼을 1,x1,x2,x1^2,x1*x2,x2^2 로 변경
df_poly.columns = ['1','x1','x2','x1^2','x1*x2','x2^2']
df_poly

,1,x1,x2,x1^2,x1*x2,x2^2
0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,2.0,3.0,4.0,6.0,9.0
2,1.0,4.0,5.0,16.0,20.0,25.0
